In [1]:
import pandas as pd
import numpy as np
import re  # 're' is the regular expression library, our most important tool here
import os

# --- Visualization for analysis ---
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# --- Load the data using our proven pathing logic ---
PROJECT_ROOT = ".." 
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "raw", "dataset")
PATH_TO_TRAIN = os.path.join(DATA_DIR, "train.csv")
PATH_TO_TEST = os.path.join(DATA_DIR, "test.csv")

# Load the datasets
train_df = pd.read_csv(PATH_TO_TRAIN)
test_df = pd.read_csv(PATH_TO_TEST)

# IMPORTANT: For consistency, we'll combine train and test for feature engineering
# This ensures that any operations are applied to both datasets equally.
# We'll add a flag to separate them later.
train_df['is_train'] = 1
test_df['is_train'] = 0
full_df = pd.concat([train_df, test_df], ignore_index=True)

# Also, let's re-create our log_price target variable
full_df['log_price'] = np.log1p(full_df['price'])


print("Data loaded and combined successfully.")
print(f"Full dataset shape: {full_df.shape}")
display(full_df.head())

Data loaded and combined successfully.
Full dataset shape: (150000, 6)


,sample_id,catalog_content,image_link,price,is_train,log_price
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,1,1.773256
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,1,2.647592
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,1,1.088562
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,1,3.444895
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,1,4.211979


In [3]:
print("--- Extracting 'Value' and 'Unit' Features ---")

# Regex to find 'Value: ' followed by a number (including decimals) or 'nan'
full_df['extracted_value'] = full_df['catalog_content'].str.extract(r'Value: ([\d\.]+|nan)')

# Regex to find 'Unit: ' followed by a word
full_df['extracted_unit'] = full_df['catalog_content'].str.extract(r'Unit: (\w+)')

# --- Data Cleaning and Type Conversion ---
full_df['extracted_value'] = pd.to_numeric(full_df['extracted_value'], errors='coerce')

# The unit is a category. Let's fill any missing ones with 'unknown' for the model.
full_df['extracted_unit'] = full_df['extracted_unit'].fillna('unknown')


print("\nExtraction complete. Let's see the results:")
# Display the new columns, plus the original text, to verify our work
display(full_df[['catalog_content', 'extracted_value', 'extracted_unit']].head())

print("\n\nLet's check the different types of units we found:")
display(full_df['extracted_unit'].value_counts())


--- Extracting 'Value' and 'Unit' Features ---

Extraction complete. Let's see the results:


,catalog_content,extracted_value,extracted_unit
0,"Item Name: La Victoria Green Taco Sauce Mild, ...",72.00,Fl
1,"Item Name: Salerno Cookies, The Original Butte...",32.00,Ounce
2,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",11.40,Ounce
3,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,11.25,Ounce
4,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",12.00,Count




Let's check the different types of units we found:


extracted_unit
Ounce         81904
Count         34911
Fl            22318
ounce          3923
None           1951
              ...  
KIT               1
NA                1
tea               1
millilitro        1
Lbs               1
Name: count, Length: 117, dtype: int64

In [8]:
print("--- Extracting Item Pack Quantity (IPQ) ---")

# A list of regex patterns to find the pack size, ordered by priority.
patterns = [
    r'(?i)pack of (\d+)',
    r'(?i)(\d+)\s*ct',
    r'(?i)(\d+)\s*count',
    r'(?i)case of (\d+)',
    r'(?i)(\d+)\s*pk',
    r'(?i)(\d+)\s*pack',
    r'(?i)set of (\d+)',
    r'(?i)(\d+)\s*bags'
]

# Create an empty Series to store our results
full_df['ipq'] = pd.Series(dtype='object') # Start as object to hold strings

# Apply each pattern and fill in the results sequentially
for pattern in patterns:
    # Extract the number (as a string) and get the first column to make it a Series
    extracted = full_df['catalog_content'].str.extract(pattern)[0]
    
    # Fill missing 'ipq' values with the newly extracted numbers
    full_df['ipq'] = full_df['ipq'].fillna(extracted)

# --- NEW SAFETY STEP: Convert to numeric and handle huge values ---
# Convert to a numeric type that can handle large numbers (float). Errors become NaN.
full_df['ipq'] = pd.to_numeric(full_df['ipq'], errors='coerce')

# Let's see what the largest pack size we found was before we clean it
print(f"Largest IPQ found before cleaning: {full_df['ipq'].max()}")

# Cap the IPQ at a reasonable maximum to handle outliers/errors (e.g., 1000)....This is a robust way to prevent errors from bad data.
IPQ_CAP = 1000
full_df.loc[full_df['ipq'] > IPQ_CAP, 'ipq'] = IPQ_CAP
print(f"Largest IPQ found AFTER cleaning: {full_df['ipq'].max()}")

# If after all our searching we still have missing values, it's a single item.
full_df['ipq'] = full_df['ipq'].fillna(1)

# Now that all values are clean and within a reasonable range, we can safely convert to integer.
full_df['ipq'] = full_df['ipq'].astype(int)


print("\nIPQ extraction complete.")
print("Let's look at some examples where we found an IPQ greater than 1:")
display(full_df[full_df['ipq'] > 1][['catalog_content', 'ipq']].head(10))

print("\n\nWhat are the most common pack sizes?")
display(full_df['ipq'].value_counts().head(10))

--- Extracting Item Pack Quantity (IPQ) ---
Largest IPQ found before cleaning: 75200007005.0
Largest IPQ found AFTER cleaning: 1000.0

IPQ extraction complete.
Let's look at some examples where we found an IPQ greater than 1:


,catalog_content,ipq
0,"Item Name: La Victoria Green Taco Sauce Mild, ...",6
1,"Item Name: Salerno Cookies, The Original Butte...",4
2,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",6
6,Item Name: Goya Foods Sazonador Total Seasonin...,6
9,Item Name: Mrs. Miller's Seedless Black Raspbe...,4
11,"Item Name: Albanese Assorted Gummi Bears, Suga...",2
12,"Item Name: KiZE Bars, 4 Count, Cookie Dough Fl...",4
13,Item Name: Smuckers Natural Peanut Butter Chun...,12
14,Item Name: BODYARMOR LYTE Sports Drink Low-Cal...,12
16,Item Name: Himalania Pink Salt Fine Jar 10.0 O...,6




What are the most common pack sizes?


ipq
1     87465
6     11262
12    10942
2      9116
3      7594
4      4832
24     3108
8      2556
10     1673
20     1115
Name: count, dtype: int64

In [9]:
print("--- Extracting Brand Name ---")

# We'll make a simple assumption: the brand is the first 1-3 words in the "Item Name".
# We can extract this text using a regular expression.
# This pattern looks for "Item Name: " and then captures the text up to the first comma, hyphen, or the words " - ".
# It's a heuristic, but it's often very effective.
brand_pattern = r'Item Name: (.*?)(,|-| - )'

# Extract the brand name using the pattern. We select the first capture group.
full_df['brand'] = full_df['catalog_content'].str.extract(brand_pattern)[0]

# --- Clean the Brand Name ---
# Remove any leading/trailing whitespace
full_df['brand'] = full_df['brand'].str.strip()
# Fill any brands we couldn't find with 'unknown'
full_df['brand'] = full_df['brand'].fillna('unknown')


print("\nBrand extraction complete.")
print("Let's see some of the brands we found:")
display(full_df[['catalog_content', 'brand']].head(10))

print("\n\nWhat are the most common brands in the dataset?")
# We'll exclude 'unknown' from the top list to see the real brands.
display(full_df[full_df['brand'] != 'unknown']['brand'].value_counts().head(15))

--- Extracting Brand Name ---

Brand extraction complete.
Let's see some of the brands we found:


,catalog_content,brand
0,"Item Name: La Victoria Green Taco Sauce Mild, ...",La Victoria Green Taco Sauce Mild
1,"Item Name: Salerno Cookies, The Original Butte...",Salerno Cookies
2,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",Bear Creek Hearty Soup Bowl
3,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,Judee’s Blue Cheese Powder 11.25 oz
4,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",kedem Sherry Cooking Wine
5,"Item Name: Member's Mark Member's Mark, Basil,...",Member's Mark Member's Mark
6,Item Name: Goya Foods Sazonador Total Seasonin...,Goya Foods Sazonador Total Seasoning
7,Item Name: VineCo Original Series Chilean Sauv...,unknown
8,Item Name: NATURES PATH CEREAL FLK MULTIGRAIN ...,NATURES PATH CEREAL FLK MULTIGRAIN ORG ECO
9,Item Name: Mrs. Miller's Seedless Black Raspbe...,unknown




What are the most common brands in the dataset?


brand
Amoretti                480
Food to Live            416
Fresh Roasted Coffee    313
Frontier Co             264
Jell                    245
Davidson's Organics     193
Amazon Brand            188
Tiesta Tea              154
Kool                    135
Cheez                   132
Amazon Fresh            123
From You Flowers        122
Monin                   118
NOW Foods                98
NUT CRAVINGS             84
Name: count, dtype: int64

In [10]:
print("--- Creating Boolean Keyword Features ---")

# Define a list of important keywords we want to check for.
# We'll use lowercase to make the search case-insensitive.
keyword_list = [
    'organic',
    'gluten free',
    'vegan',
    'non-gmo',
    'kosher',
    'natural',
    'gourmet',
    'premium'
]

# Loop through each keyword and create a new column for it
for keyword in keyword_list:
    # Create a new column name, e.g., 'is_organic'
    col_name = f'is_{keyword.replace(" ", "_")}'
    
    # Search the lowercase version of the catalog_content for the keyword.
    # This returns True or False for each row. We convert it to 1 or 0 for the model.
    full_df[col_name] = full_df['catalog_content'].str.lower().str.contains(keyword).astype(int)

print("\nKeyword feature creation complete.")
print("Let's see the new columns for a few of our products:")

# Create a list of our new columns to display them easily
new_keyword_cols = [f'is_{kw.replace(" ", "_")}' for kw in keyword_list]
display(full_df[['catalog_content'] + new_keyword_cols].head())

print("\n\nHow many products have each of these attributes?")
# This will show us how common each keyword is in the dataset.
display(full_df[new_keyword_cols].sum().sort_values(ascending=False))

--- Creating Boolean Keyword Features ---

Keyword feature creation complete.
Let's see the new columns for a few of our products:


,catalog_content,is_organic,is_gluten_free,is_vegan,is_non-gmo,is_kosher,is_natural,is_gourmet,is_premium
0,"Item Name: La Victoria Green Taco Sauce Mild, ...",0,0,0,0,0,0,0,0
1,"Item Name: Salerno Cookies, The Original Butte...",0,0,0,0,0,0,0,0
2,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",0,0,0,0,0,0,0,0
3,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,0,0,0,0,0,0,0,0
4,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",0,0,0,0,0,0,0,0




How many products have each of these attributes?


is_natural        38373
is_kosher         26430
is_premium        24098
is_organic        20862
is_gluten_free    19852
is_non-gmo        18982
is_vegan          16820
is_gourmet        10242
dtype: int64

In [11]:
print("--- Creating Text Statistics Features ---")

# Let's analyze the raw text content itself.
# We will use the full catalog_content for this.
text_col = 'catalog_content'

# 1. Length of the description (number of characters)
full_df['text_len'] = full_df[text_col].str.len()

# 2. Number of words in the description
full_df['word_count'] = full_df[text_col].str.split().str.len()

# 3. Average length of words
# We'll add 1e-6 to the denominator to avoid dividing by zero if a description is empty.
full_df['avg_word_len'] = full_df['text_len'] / (full_df['word_count'] + 1e-6)

# 4. Number of unique words (a measure of vocabulary richness)
full_df['unique_word_count'] = full_df[text_col].apply(lambda x: len(set(str(x).split())))


print("\nText statistics features created.")
print("Let's see how these new features correlate with our target, log_price.")

# Calculate the correlation of our new features with the log_price
# We only look at the training data for this, as the test data has no price.
correlation_cols = ['log_price', 'text_len', 'word_count', 'avg_word_len', 'unique_word_count']
correlation_matrix = full_df[full_df['is_train'] == 1][correlation_cols].corr()

# Display the correlation of each feature with log_price
print("\nCorrelation with log_price:")
display(correlation_matrix['log_price'].sort_values(ascending=False))

--- Creating Text Statistics Features ---

Text statistics features created.
Let's see how these new features correlate with our target, log_price.

Correlation with log_price:


log_price            1.000000
text_len             0.258967
word_count           0.255328
unique_word_count    0.249068
avg_word_len         0.100926
Name: log_price, dtype: float64

In [13]:
# --- Save the Processed DataFrame ---
# We will save our work to the 'processed' data folder.
# Using the Parquet format is much more efficient than CSV for saving data with mixed types.
PROCESSED_DIR = os.path.join(PROJECT_ROOT, "data", "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True) # Ensure the directory exists

# Defining the path for our feature-engineered file
FEATURES_PATH = os.path.join(PROCESSED_DIR, "features_v1.parquet")

# Selecting all columns EXCEPT the original text and image link, which we won't need for the baseline model
cols_to_save = [col for col in full_df.columns if col not in ['catalog_content', 'image_link']]

# Saving the data
full_df[cols_to_save].to_parquet(FEATURES_PATH, index=False)

print(f"✅ Feature-engineered data saved successfully to:\n{FEATURES_PATH}")
print(f"The saved data has {full_df.shape[0]} rows and {len(cols_to_save)} columns.")

✅ Feature-engineered data saved successfully to:
..\data\processed\features_v1.parquet
The saved data has 150000 rows and 20 columns.
